# Underfitting and Overfitting. Fine-tune your model for better perfomance.

Al final de este paso entenderás los conceptos de **underfitting** y **overfitting**, y sabrás cómo aplicar estas ideas para hacer que tus modelos sean más precisos.


<div style="background-color:#FFF4CC; padding:12px; border-radius:4px;">


### Experimentar con distintos modelos

Ahora que tienes una forma fiable de medir la precisión de un modelo, puedes experimentar con modelos alternativos y comprobar cuál ofrece mejores predicciones. Pero, ¿qué alternativas tienes?

En la documentación de **scikit-learn** puedes ver que el modelo de **decision tree** tiene muchas opciones (más de las que necesitarás durante bastante tiempo). Las opciones más importantes determinan la **profundidad del árbol**.

Recuerda de la primera lección que la profundidad de un árbol indica cuántas divisiones (*splits*) realiza antes de llegar a una predicción.  


</div>


<div style="background-color:#FFF4CC; padding:12px; border-radius:4px;">

En la práctica, no es raro que un árbol tenga unas **10 divisiones (splits)** entre el nivel superior (todas las viviendas) y una hoja. A medida que el árbol se hace más profundo, el dataset se va dividiendo en hojas que contienen cada vez **menos viviendas**.

Si un árbol tuviera solo **1 split**, dividiría los datos en **2 grupos**.  
Si cada uno de esos grupos se divide otra vez, obtendríamos **4 grupos**.  
Si volvemos a dividir cada uno, tendríamos **8 grupos**.  

Si seguimos duplicando el número de grupos añadiendo más splits en cada nivel, al llegar al nivel 10 tendríamos:

2¹⁰ = **1024 grupos (hojas)**.

Cuando dividimos las viviendas en tantas hojas, cada hoja contiene muy pocas casas. Las hojas con muy pocos datos hacen predicciones muy cercanas a los valores reales de esas casas concretas, pero suelen ser **muy poco fiables para datos nuevos**, porque cada predicción se basa en muy pocos ejemplos.

Este fenómeno se llama **overfitting**: el modelo se ajusta casi perfectamente a los datos de entrenamiento, pero funciona mal con datos de validación o datos nuevos.

En el extremo contrario, si hacemos el árbol **muy poco profundo**, no divide las viviendas en grupos suficientemente distintos.

Por ejemplo, si el árbol solo divide las casas en **2 o 4 grupos**, cada grupo sigue conteniendo viviendas muy diferentes entre sí. Como resultado, las predicciones pueden ser muy malas para la mayoría de las casas, incluso en los datos de entrenamiento (y también en validación, por la misma razón).

Cuando un modelo no consigue capturar patrones y diferencias importantes en los datos y funciona mal incluso con los datos de entrenamiento, hablamos de **underfitting**.

Como lo que realmente nos importa es la precisión en **datos nuevos**, que estimamos usando los datos de validación, buscamos un punto intermedio entre **underfitting** y **overfitting**. Visualmente, queremos encontrar el punto más bajo de la curva de validación (la curva roja) en la figura.

</div>


<img src="assets/1.png" alt="imagen" style="max-width:50%;">


<div style="background-color:#FFF4CC; padding:12px; border-radius:4px;">

### Ejemplo

Existen varias alternativas para controlar la profundidad de un árbol, y muchas de ellas permiten que algunas ramas del árbol tengan mayor profundidad que otras. Sin embargo, el argumento **max_leaf_nodes** ofrece una forma muy sensata de controlar el equilibrio entre **overfitting** y **underfitting**.

Cuantas más hojas permitimos que tenga el modelo, más nos desplazamos desde la zona de **underfitting** hacia la zona de **overfitting** en la gráfica anterior.

Podemos usar una **función auxiliar** para comparar los valores de **MAE** obtenidos con distintos valores de **max_leaf_nodes**.

</div>


In [6]:
from sklearn.metrics import mean_absolute_error
from sklearn.tree import DecisionTreeRegressor

In [11]:
def obtener_mae(numero_nodos, train_X, val_X, train_y, val_y):
    modelo=DecisionTreeRegressor(max_leaf_nodes=numero_nodos,random_state=0)
    modelo.fit(train_X,train_y)
    prediccion=modelo.predict(val_X)
    mae=mean_absolute_error(val_y,prediccion)
    return(mae)

<div style="background-color:#FFF4CC; padding:12px; border-radius:4px;">

Los datos se cargan en **train_X**, **val_X**, **train_y** y **val_y** usando el código que ya has visto (y que ya has escrito anteriormente).

</div>


In [10]:
# Data Loading Code Runs At This Point
import pandas as pd
    
# Load data
melbourne_file_path = 'data/melb_data.csv'
melbourne_data = pd.read_csv(melbourne_file_path) 
# Filter rows with missing values
filtered_melbourne_data = melbourne_data.dropna(axis=0)
# Choose target and features
y = filtered_melbourne_data.Price
melbourne_features = ['Rooms', 'Bathroom', 'Landsize', 'BuildingArea', 
                        'YearBuilt', 'Lattitude', 'Longtitude']
X = filtered_melbourne_data[melbourne_features]

from sklearn.model_selection import train_test_split

# split data into training and validation data, for both features and target
train_X, val_X, train_y, val_y = train_test_split(X, y,random_state = 0)

<div style="background-color:#FFF4CC; padding:12px; border-radius:4px;">

Podemos usar un **bucle for** para comparar la precisión de modelos construidos con distintos valores de **max_leaf_nodes**.

</div>


In [16]:
# Comparando MAE con diferentes valores de max_leaf_nodes
for max_leaf_nodes in [5,50,500,5000]:
    my_mae=obtener_mae(max_leaf_nodes, train_X,val_X,train_y, val_y)
    print("Max leaf nodes: %d  \t\t Mean Absolute Error:  %d" %(max_leaf_nodes, my_mae))
    print(" ")
    
    #print(f"Numero de nodos: {max_leaf_nodes} | Mean Absolute Error = {my_mae}")
    

Max leaf nodes: 5  		 Mean Absolute Error:  347380
 
Max leaf nodes: 50  		 Mean Absolute Error:  258171
 
Max leaf nodes: 500  		 Mean Absolute Error:  243495
 
Max leaf nodes: 5000  		 Mean Absolute Error:  255575
 


<div style="background-color:#FFF4CC; padding:12px; border-radius:4px;">

De las opciones analizadas, **500** es el número óptimo de hojas.

</div>


<p align="center">
  <img src="assets/separador.png" alt="Separador" width=300"/>
</p>

<div style="background-color:#FFF4CC; padding:12px; border-radius:4px;">

Conclusión

La idea clave es la siguiente: los modelos pueden sufrir uno de estos dos problemas:

- **Overfitting**: capturan patrones espurios que no se repetirán en el futuro, lo que conduce a predicciones menos precisas.
- **Underfitting**: no consiguen capturar patrones relevantes, lo que también provoca predicciones poco precisas.

Para medir la precisión real de un modelo usamos **validation data**, que no se utiliza durante el entrenamiento. Esto nos permite probar muchos modelos candidatos y quedarnos con el que ofrece el mejor rendimiento.

</div>
